In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import json

dataname = "sachs"
groundtruth = np.array(json.load(open(f"../CausalBKAI/data/TestData/bnlearn_discrete_10000/truth_dag_adj/{dataname}.json", "r"))['Adj'])

mi = 3      # The number of values a variable can take is ranged in [2, mi-1]
di = 1      # The dirichlet alpha that controls the data distribution
n = 10      # The number of data silos

silos = []

folderpath = f"./data/distributed/{dataname}/m{mi}_d{di}_n{n}"

if not Path(folderpath).exists():
    print("Folder", folderpath, "not exist!")
else:
    for file in sorted(os.listdir(folderpath)):
        filename = os.path.join(folderpath, file)
        silo_data = pd.read_csv(filename)
        silos.append(silo_data)
        print("Loaded file:", filename, end="\t")
        all_vars = silos[0].columns
        print(len(silo_data), " Instances\t", len(all_vars), "Variables")
        

merged_df = pd.concat(silos, axis=0)

Loaded file: ./data/distributed/sachs/m3_d1_n10/silo-0.csv	10000  Instances	 11 Variables
Loaded file: ./data/distributed/sachs/m3_d1_n10/silo-1.csv	10000  Instances	 11 Variables
Loaded file: ./data/distributed/sachs/m3_d1_n10/silo-2.csv	10000  Instances	 11 Variables
Loaded file: ./data/distributed/sachs/m3_d1_n10/silo-3.csv	10000  Instances	 11 Variables
Loaded file: ./data/distributed/sachs/m3_d1_n10/silo-4.csv	10000  Instances	 11 Variables
Loaded file: ./data/distributed/sachs/m3_d1_n10/silo-5.csv	10000  Instances	 11 Variables
Loaded file: ./data/distributed/sachs/m3_d1_n10/silo-6.csv	10000  Instances	 11 Variables
Loaded file: ./data/distributed/sachs/m3_d1_n10/silo-7.csv	10000  Instances	 11 Variables
Loaded file: ./data/distributed/sachs/m3_d1_n10/silo-8.csv	10000  Instances	 11 Variables
Loaded file: ./data/distributed/sachs/m3_d1_n10/silo-9.csv	10000  Instances	 11 Variables


#### Constraint-based PC

In [2]:
pd.__version__

'1.5.3'

In [3]:
import bnlearn as bn

model = bn.structure_learning.fit(merged_df, methodtype='cs', verbose=0)

adj_mtx = np.zeros([len(all_vars), len(all_vars)])
for edge in model['dag_edges']:     # type:ignore
    source, target = edge
    source_id = int(source[1:]) - 1
    target_id = int(target[1:]) - 1
    adj_mtx[source_id][target_id] = 1

  0%|          | 0/5 [00:00<?, ?it/s]

[datazets] >INFO> Reached maximum number of allowed conditional variables. Exiting
[datazets] >WARNING> PDAG has no faithful extension (= no oriented DAG with the same v-structures as PDAG). Remaining undirected PDAG edges oriented arbitrarily.


  0%|          | 0/5 [00:00<?, ?it/s]

[datazets] >INFO> Reached maximum number of allowed conditional variables. Exiting
[datazets] >WARNING> PDAG has no faithful extension (= no oriented DAG with the same v-structures as PDAG). Remaining undirected PDAG edges oriented arbitrarily.


In [4]:
from plot_utils import true_edge, spur_edge, fals_edge, miss_edge, swap_pos

etrue = true_edge(groundtruth, adj_mtx)
espur = spur_edge(groundtruth, adj_mtx)
efals = fals_edge(groundtruth, adj_mtx)
emiss = miss_edge(groundtruth, adj_mtx)

# print(etrue)
print(len(etrue), len(espur), len(emiss), len(efals))

6 0 2 9


#### Causal discovery from nonstationary/heterogeneous data (CDNOD)

In [8]:
from causallearn.search.ConstraintBased.CDNOD import cdnod
from baselines.FL_FedCDH.mycausallearn.utils.data_utils import get_cpdag_from_cdnod, get_dag_from_pdag
from causallearn.utils.cit import fisherz

c_indx = []
for i in range(len(silos)):
    data = silos[i]
    c_indx += [i+1] * len(data)
c_indx = np.array(c_indx).reshape(len(silos)*len(silos[0]), 1)

cg = cdnod(merged_df.to_numpy(), c_indx, 0.05, fisherz)

est_graph = cg.G.graph[0:len(all_vars), 0:len(all_vars)]
est_cpdag = get_cpdag_from_cdnod(est_graph) # est_graph[i,j]=-1 & est_graph[j,i]=1  ->  est_graph_cpdag[i,j]=1
est_dag_from_pdag = get_dag_from_pdag(est_cpdag) # return a DAG from a PDAG in causaldag

  0%|          | 0/12 [00:00<?, ?it/s]

In [9]:
from plot_utils import true_edge, spur_edge, fals_edge, miss_edge

etrue = true_edge(groundtruth, est_dag_from_pdag)
espur = spur_edge(groundtruth, est_dag_from_pdag)
efals = fals_edge(groundtruth, est_dag_from_pdag)
emiss = miss_edge(groundtruth, est_dag_from_pdag)

# print(etrue)
print(len(etrue), len(espur), len(emiss), len(efals))

5 13 10 2


#### Constraint-based Fast Causal Inference (FCI)

In [50]:
from causallearn.search.ConstraintBased.FCI import fci

# default parameters
g, edges = fci(merged_df.to_numpy())

  0%|          | 0/37 [00:00<?, ?it/s]

X6 --> X3
X3 --> X7
X3 --> X9
X3 --> X11
X12 --> X3
X6 --> X7
X9 --> X6
X11 --> X6
X12 --> X6
X6 --> X16
X9 --> X7
X11 --> X7
X12 --> X7
X11 --> X9
X9 --> X12
X9 --> X17
X9 --> X18
X12 --> X19
X14 --> X20
X14 --> X21
X15 --> X36
X16 --> X31
X16 --> X37
X18 --> X30
X22 --> X23
X26 --> X27
X26 --> X28
X27 --> X30
X28 --> X29
X28 --> X31
X32 --> X35


In [51]:
# for edge in edges:
#     print(edge, "\t", edge.get_node1(), edge.get_numerical_endpoint1(), edge.get_node2(), edge.get_numerical_endpoint2())

adj_mtx = np.zeros([len(all_vars), len(all_vars)])
for edge in edges:     # type:ignore
    source_id = int(edge.get_node1().get_name()[1:]) - 1
    target_id = int(edge.get_node2().get_name()[1:]) - 1
    
    if edge.get_numerical_endpoint1() == -1:
        adj_mtx[source_id][target_id] = 1
    elif edge.get_numerical_endpoint1() != 2:
        adj_mtx[target_id][source_id] = 1

In [52]:
from plot_utils import true_edge, spur_edge, fals_edge, miss_edge, swap_pos

etrue = true_edge(groundtruth, adj_mtx)
espur = spur_edge(groundtruth, adj_mtx)
efals = fals_edge(groundtruth, adj_mtx)
emiss = miss_edge(groundtruth, adj_mtx)

# print(etrue)
print(len(etrue), len(espur), len(emiss), len(efals))

4 61 34 8


#### Value-based HillClimb GES

In [61]:
import bnlearn as bn

model = bn.structure_learning.fit(merged_df, methodtype='hc', verbose=0)
adj_mtx = model['adjmat'].to_numpy() * 1.0      # type:ignore

In [62]:
from plot_utils import true_edge, spur_edge, fals_edge, miss_edge

etrue = true_edge(groundtruth, adj_mtx.T)
espur = spur_edge(groundtruth, adj_mtx.T)
efals = fals_edge(groundtruth, adj_mtx.T)
emiss = miss_edge(groundtruth, adj_mtx.T)

# print(etrue)
print(len(etrue), len(espur), len(emiss), len(efals))

4 103 70 10


#### Value-based Treesearch Chow-Liu Algorithm

In [ ]:
import bnlearn as bn
# Preprocessing raw dataset
dfhot, dfnum = bn.df2onehot(merged_df)

# Structure learning
model = bn.structure_learning.fit(dfnum, methodtype='cl', verbose=0)
adj_mtx = model['adjmat'].to_numpy() * 1.0      # type:ignore

In [ ]:
from plot_utils import true_edge, spur_edge, fals_edge, miss_edge

etrue = true_edge(groundtruth, adj_mtx)
espur = spur_edge(groundtruth, adj_mtx)
efals = fals_edge(groundtruth, adj_mtx)
emiss = miss_edge(groundtruth, adj_mtx)

# print(etrue)
print(len(etrue), len(espur), len(emiss), len(efals))

#### Value-based Tree-augmented Naive Bayes (TAN) 

In [ ]:
import bnlearn as bn

# Structure learning
model = bn.structure_learning.fit(merged_df, methodtype='tan', class_node='X1', verbose=0)
pruned_model = bn.independence_test(model, merged_df, alpha=0.05, prune=True)

adj_mtx = pruned_model['adjmat'].to_numpy() * 1.0      # type:ignore

In [ ]:
from plot_utils import true_edge, spur_edge, fals_edge, miss_edge

etrue = true_edge(groundtruth, adj_mtx)
espur = spur_edge(groundtruth, adj_mtx)
efals = fals_edge(groundtruth, adj_mtx)
emiss = miss_edge(groundtruth, adj_mtx)

# print(etrue)
print(len(etrue), len(espur), len(emiss), len(efals))

### CDT.CAUSALITY

#### Concave penalized Coordinate Descent with reparametrization (CCDR)

In [ ]:
from cdt.causality.graph import CCDr

#### Greedy Interventional Equivalence Search algorithm (GIES)

In [ ]:
from cdt.causality.graph import GIES

#### Structural Agnostic Model

In [ ]:
from cdt.causality.graph import SAM

#### Causal Generative Neural Networks

In [ ]:
from cdt.causality.graph import CGNN